<a href="https://colab.research.google.com/github/Ajay-Sai-Kiran/GANS/blob/main/DAY_3_Deep_Convolution_GAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Types of GANS

#Foundation

Generative Adversarial Network (GAN)

Deep Convolutional Generative Adversarial Network (DCGAN)

#Extensions
Conditional Generative Adversarial Network (cGAN)

Information Maximizing Generative Adversarial Network (InfoGAN)

Auxiliary Classifier Generative Adversarial Network (AC-GAN)
Stacked Generative Adversarial Network (StackGAN)


Context Encoders

Pix2Pix

#Advanced

Wasserstein Generative Adversarial Network (WGAN)

Cycle-Consistent Generative Adversarial Network (CycleGAN)

Progressive Growing Generative Adversarial Network (Progressive GAN)

Style-Based Generative Adversarial Network (StyleGAN)

Big Generative Adversarial Network (BigGAN)


#Deep Convolutional Generative Adversarial Network (DCGAN)

The deep convolutional generative adversarial network, or DCGAN for short, is an extension of the GAN architecture for using deep convolutional neural networks for both the generator and discriminator models and configurations for the models and training that result in the stable training of a generator model

DCGAN uses **convolutional and convolutional-transpose layers** in the generator and discriminator, respectively.

Here the **discriminator consists of strided convolution layers, batch normalization layers, and LeakyRelu as activation function**. It takes a 3x64x64 input image. 

The role of the discriminator here is to determine that the image comes from either real dataset or generator.   The discriminator can be simply designed similar to a convolution neural network that performs a image classification task.



The **generator consists of convolutional-transpose layers, batch normalization layers, and ReLU activations**. The output will be a 3x64x64 RGB image.

The generator of the DCGAN architecture takes 100 uniform generated values using normal distribution as an input. 

First, it changes the dimension  to 4x4x1024 and performed a fractionally strided convolution in 4 times with stride of 1/2 (this means every time when applied, it doubles the image dimension while reducing the number of output channels). 

The generated output has dimensions of (64, 64, 3). There are some architectural changes proposed in generator such as removal of all fully connected layer, use of Batch Normalization which helps in stabilizing training.

#Implementation

In [ ]:
# code % matplotlib inline
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython import display

# Check tensorflow version
print('Tensorflow version:', tf.__version__)


we load the fashion-MNIST dataset, the good thing is that dataset can be imported  from tf.keras.datasets API. So, we don’t need to load datasets manually by copying files. This dateset contains 60k training images and 10k test images each of dimensions(28, 28, 1). Since the value of each pixel is in the range (0, 255), we divide these values by 255 to normalize it.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train.shape, x_test.shape


we will  be visualizing some of the images from Fashion-MNIST dateset, we use matplotlib library for that.

In [ ]:
# We plot first 25 images of training dataset
plt.figure(figsize =(10, 10))
for i in range(25):
	plt.subplot(5, 5, i + 1)
	plt.xticks([])
	plt.yticks([])
	plt.grid(False)
	plt.imshow(x_train[i], cmap = plt.cm.binary)
plt.show()


we define training parameters such as batch size and divides the dataset into batch size and fills those batch size by randomly sampling the training data.

In [ ]:
# code
batch_size = 32
# This dataset fills a buffer with buffer_size elements,
# then randomly samples elements from this buffer,
# replacing the selected elements with new elements.
def create_batch(x_train):
  dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
# Combines consecutive elements of this dataset into batches.

  dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1)
# Creates a Dataset that prefetches elements from this dataset
  return dataset


we define the generator architecture, this generator architecture takes a vector of size 100 and first reshape that into (7, 7, 128) vector then applied transpose convolution in combination with batch normalization. The output of this generator is a trained an image of dimension (28, 28, 1).

The strided conv-transpose layers allow the latent vector to be transformed into a volume with the same shape as an image.

In [ ]:

num_features = 100

generator = keras.models.Sequential([
	keras.layers.Dense(7 * 7 * 128, input_shape =[num_features]),
	keras.layers.Reshape([7, 7, 128]),
	keras.layers.BatchNormalization(),
	keras.layers.Conv2DTranspose(
		64, (5, 5), (2, 2), padding ="same", activation ="selu"),
	keras.layers.BatchNormalization(),
	keras.layers.Conv2DTranspose(
		1, (5, 5), (2, 2), padding ="same", activation ="tanh"),
])
generator.summary()


we define out discriminator architecture, the discriminator takes image of size  28*28 with 1 color channel and output a scalar value representing image from either dataset or generated image.

In [ ]:
discriminator = keras.models.Sequential([
	keras.layers.Conv2D(64, (5, 5), (2, 2), padding ="same", input_shape =[28, 28, 1]),
	keras.layers.LeakyReLU(0.2),
	keras.layers.Dropout(0.3),
	keras.layers.Conv2D(128, (5, 5), (2, 2), padding ="same"),
	keras.layers.LeakyReLU(0.2),
	keras.layers.Dropout(0.3),
	keras.layers.Flatten(),
	keras.layers.Dense(1, activation ='sigmoid')
])
discriminator.summary()


we need to compile the our DCGAN model (combination of generator and discriminator), we will first compile discriminator and set its  training to False, because we first want to train the generator.

In [ ]:
# compile discriminator using binary cross entropy loss and adam optimizer
discriminator.compile(loss ="binary_crossentropy", optimizer ="adam")
# make discriminator no-trainable as of now
discriminator.trainable = False
# Combine both generator and discriminator
gan = keras.models.Sequential([generator, discriminator])
# compile generator using binary cross entropy loss and adam optimizer

gan.compile(loss ="binary_crossentropy", optimizer ="adam")


we define the training procedure for this GAN model, we will be using tqdm package which we have imported earlier., this package help in visualizing training.

In [ ]:
seed = tf.random.normal(shape =[batch_size, 100])

def train_dcgan(gan, dataset, batch_size, num_features, epochs = 5):
	generator, discriminator = gan.layers
	for epoch in tqdm(range(epochs)):
		print()
		print("Epoch {}/{}".format(epoch + 1, epochs))

		for X_batch in dataset:
			# create a random noise of sizebatch_size * 100
			# to passit into the generator
			noise = tf.random.normal(shape =[batch_size, num_features])
			generated_images = generator(noise)

			# take batch of generated image and real image and
			# use them to train the discriminator
			X_fake_and_real = tf.concat([generated_images, X_batch], axis = 0)
			y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
			discriminator.trainable = True
			discriminator.train_on_batch(X_fake_and_real, y1)

			# Here we will be training our GAN model, in this step
			# we pass noise that uses geeneratortogenerate the image
			# and pass it with labels as [1] So, it can fool the discriminatoe
			noise = tf.random.normal(shape =[batch_size, num_features])
			y2 = tf.constant([[1.]] * batch_size)
			discriminator.trainable = False
			gan.train_on_batch(noise, y2)

			# generate images for the GIF as we go
			generate_and_save_images(generator, epoch + 1, seed)

	generate_and_save_images(generator, epochs, seed)


we define a function that generate and save images from generator (during training). We will use these generated images to plot the GIF later.

In [ ]:
# code
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training = False)

  fig = plt.figure(figsize =(10, 10))

  for i in range(25):
	  plt.subplot(5, 5, i + 1)
	  plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap ='binary')
	  plt.axis('off')

  plt.savefig('image_epoch_{:04d}.png'.format(epoch))


we need to train the model but before that we also need to create batches of training data and add a dimension that represents number  of color maps.

In [ ]:
# reshape to add a color map
x_train_dcgan = x_train.reshape(-1, 28, 28, 1) * 2. - 1.
# create batches
dataset = create_batch(x_train_dcgan)
# callthe training function with 10 epochs and record time %% time
train_dcgan(gan, dataset, batch_size, num_features, epochs = 10)


In [ ]:
import imageio
import glob

anim_file = 'dcgan_results.gif'

with imageio.get_writer(anim_file, mode ='I') as writer:
filenames = glob.glob('image*.png')
filenames = sorted(filenames)
last = -1
for i, filename in enumerate(filenames):
	frame = 2*(i)
	if round(frame) > round(last):
	last = frame
	else:
	continue
	image = imageio.imread(filename)
	writer.append_data(image)
image = imageio.imread(filename)
writer.append_data(image)
display.Image(filename = anim_file)


#Reference:
Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks

by Alec Radford, Luke Metz, Soumith Chintala